# Automating FOTMP Review - Second Session

Continuing the lessons from last week, let's go over the next check that is on our list. This check will be a bit more involved than the first on.

<b>Check #2</b></br>
<i>Verify no format commanding during ACIS bias: egrep "ACIS,XTZ*|ACIS,XCZ*|ATS,TLM,FORMAT=*" *dot</i></br></br>

The check includes a <b>grep</b> that can be used to do this check manually, which output all commands in the DOT that include the XTZ*, XCZ*, and FORMAT statements in them. This will need a reviewer to look at each instance and make an analysis if there is an issue or not. What we want is to be able output the information to the user so a quick analysis can be done by only looking at the trouble area. Luckily, python can help with this!!

## Thinking about the logic

First and foremost, can we get away from using the DOT? The goal here is to be to use less files in the anlysis as possible, so let's see if this check can be done from a backstop file. Snippet of a backstop file with a with pertinent information shown.Keep in mind that this is an abridged version and does not reflect a full backstop file.

<font color=blue>
2018:078:21:34:19.857 | 16603512 0 | ACISPKT          | TLMSID= AA00000000<br>
2018:078:21:34:23.857 | 16603528 0 | ACISPKT          | TLMSID= AA00000000<br>
2018:078:21:34:27.857 | 16603544 0 | ACISPKT          | TLMSID= WSPOW00000<br>
2018:078:21:34:51.857 | 16603637 0 | ACISPKT          | TLMSID= WSPOW3F03F<br>
2018:078:21:35:54.857 | 16603883 0 | ACISPKT          | TLMSID= WT0092E024<br>
**2018:078:21:35:58.857 | 16603899 0 | ACISPKT          | TLMSID= XTZ0000005**<br>
2018:078:21:36:02.857 | 16603914 0 | ACISPKT          | TLMSID= RS_0000001<br>
2018:078:21:36:06.857 | 16603930 0 | ACISPKT          | TLMSID= RH_0000001<br>



**2018:079:10:39:19.069 |    10098 0 | COMMAND_HW       | TLMSID= CSELFMT4**<br>
2018:079:10:39:19.069 |    10098 0 | MP_OBSID         | TLMSID= COAOSQID<br>


2018:079:10:39:53.408 |    10232 0 | COMMAND_SW       | TLMSID= 4OHETGRE<br>
2018:079:10:43:23.408 |    11052 0 | COMMAND_SW       | TLMSID= 4OOTGDS<br>
2018:079:10:43:24.433 |    11056 0 | COMMAND_HW       | TLMSID= 4MC5ADS<br>
2018:079:10:43:36.369 |    11102 0 | COMMAND_HW       | TLMSID= 2NXILASL<br>
**2018:079:10:43:36.869 |    11104 0 | COMMAND_HW       | TLMSID= CSELFMT2**<br>
</font>


From the looks of it, we should be able to use the backstop file to do this check. All the points we are looking for are located in the file, so it would be a good idea to move this check to this file.

In [35]:
from re import search
from glob import glob
from datetime import datetime

In [40]:
def time2seconds(timestring):
    parsedtime = datetime.strptime(timestring,"%Y:%j:%H:%M:%S.%f") - datetime(1998,1,1)
    return parsedtime.total_seconds()

In [41]:
backstop_file_array = glob('/home/mission/Backstop/MAR1218B/CR*backstop')
backstop_file_array

['/home/mission/Backstop/MAR1218B/CR071_0604.backstop']

Let's do a little error checking to make sure there is only one file and check to see if there is a backstop file to use.

In [42]:
if len(backstop_file_array) > 1:
    print("There are too many backstop files")
    
    for backstop_file in backstop_file_array:
        bstop_array = backstop_file.split('/')
        print(bstop_array[-1])
    
if len(backstop_file_array) == 0:
    print("There is no backstop file")

In a perfect, we would never need this check, but it is good to think about these type of situations as we move forward. Now let's open the file to get it ready to read.

In [67]:
file_in = open(backstop_file_array[0])
# "(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (XCZ[0-9,A-Z]+)",
regex_to_match = ["(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (X[T C]Z[0-9 A-F]+)",
                  "(\d+:\d+:\d+:\d+:\d+.\d+).*TLMSID= (CSELFMT\d)"]

results_array = []



for line in file_in:
    match = search(regex_to_match[0],line)
    if not match:
        match = search(regex_to_match[1],line)
    if match:
        temp_object = {
            "timestring": match.group(1),
            "tlmsid":match.group(2),
            "time_seconds":time2seconds(match.group(1))
        }
        
        results_array.append(temp_object)
        
        
        print("TIME: "+ match.group(1) + " TLMSID: " + match.group(2))

#for result_instance in results_array:
#    print("TIME: "+ result_instance['time'] + " TLMSID: " + result_instance['tlmsid'])

file_in.close()

TIME: 2018:071:17:07:55.192 TLMSID: XTZ0000005
TIME: 2018:071:18:31:12.192 TLMSID: XTZ0000005
TIME: 2018:071:22:56:24.192 TLMSID: XTZ0000005
TIME: 2018:072:06:17:32.293 TLMSID: XTZ0000005
TIME: 2018:072:23:53:06.293 TLMSID: CSELFMT4
TIME: 2018:072:23:57:24.093 TLMSID: CSELFMT2
TIME: 2018:073:00:03:01.914 TLMSID: XTZ0000005
TIME: 2018:073:05:06:43.914 TLMSID: CSELFMT4
TIME: 2018:073:05:11:01.714 TLMSID: CSELFMT2
TIME: 2018:073:05:14:47.535 TLMSID: XTZ0000005
TIME: 2018:073:14:18:19.267 TLMSID: XTZ0000005
TIME: 2018:073:16:23:36.267 TLMSID: XTZ0000005
TIME: 2018:073:19:42:18.432 TLMSID: XTZ0000005
TIME: 2018:074:02:46:51.293 TLMSID: CSELFMT4
TIME: 2018:074:02:51:09.093 TLMSID: CSELFMT2
TIME: 2018:074:03:53:26.000 TLMSID: CSELFMT6
TIME: 2018:074:03:57:53.000 TLMSID: CSELFMT2
TIME: 2018:074:08:40:58.432 TLMSID: XTZ0000005
TIME: 2018:074:10:04:15.432 TLMSID: XTZ0000005
TIME: 2018:074:18:45:59.432 TLMSID: CSELFMT4
TIME: 2018:074:18:50:17.232 TLMSID: CSELFMT2
TIME: 2018:074:18:52:29.032 TLMSI

In [68]:
previous_object = {}

index=0
for result_instance in results_array:
    if not index == 0:
        time = result_instance['time_seconds']
        previous_time = previous_object['time_seconds']
        
        match=search("SELFMT\d",result_instance['tlmsid'])
        if match:
            match2=search("SELFMT\d",previous_object['tlmsid'])
            
            if not match2:
                if  previous_time+(26*60) > time:
                    output = "TLMSID: "+ result_instance['tlmsid']+" "+result_instance['timestring'] +\
                    " | PREVIOUS TLMSID: "+ previous_object['tlmsid'] +" " +\
                    previous_object['timestring'] + "|<------THIS NEED CHECKED | AMOUNT of TIME BETWEEN: "+\
                    str((result_instance['time_seconds']-previous_object['time_seconds'])/60)
                    
                    print(output)
                
    
    index+=1
    previous_object = result_instance
    
    
    